In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [3]:
COLUMNS = ['segment_id', 'split_index', 'total_end_y']

#### Load data

In [4]:
df_neuron = pd.read_pickle('../../../data/data_tables/df_classifier.pkl')
df_morphos = pd.read_pickle('../../../data/data_tables/df_morphos.pkl')
df = pd.merge(df_neuron, df_morphos[COLUMNS], on=['segment_id', 'split_index'])

In [5]:
df_l4 = df[df['assigned_layer'] == 'L4']

In [6]:
len(df_l4)

8332

In [7]:
df_l4.loc[:, 'exclude'] = False

/tmp/ipykernel_1900638/2047010260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l4.loc[:, 'exclude'] = False


#### Exclude L4 neurons with apical dendrite ending below threshold

In [8]:
# Threshold determined in neuron_preprocessing
treshold = 96.0

In [9]:
assert df_l4['total_end_y'].isna().sum() == 0

In [10]:
df_l4['total_end_y'].min(), df_l4['total_end_y'].max()

(-38.66239032035219, 292.688123774083)

In [11]:
df_l4.loc[df_l4['total_end_y'] > treshold, 'exclude'] = True
df_l4.loc[df_l4['total_end_y'] > treshold, 'exclusion_criterion'] = (
    'apical_end_above_threshold'
)

percent = df_l4.exclude.value_counts(normalize=True)[1] * 100
print(f'L4 exclusion: {percent:.0f}%; counts:\n{df_l4.exclude.value_counts()}.')

L4 exclusion: 7%; counts:
False    7774
True      558
Name: exclude, dtype: int64.


/tmp/ipykernel_1900638/1405428351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l4.loc[df_l4['total_end_y'] > treshold, 'exclusion_criterion'] = (


In [12]:
df = pd.concat([df[df.assigned_layer != 'L4'], df_l4])

In [13]:
len(df)

43666